In [ ]:
import warnings
warnings.filterwarnings('ignore')

# About this notebook

Author: Anubhav Jain

Github repo: TBD

YouTube video: TBD

![Matcalc tutorial](graphics/title.png "Matcalc Tutorial")

# Tutorial overview

In this tutorial, we are going to learn to very quickly simulate materials properties such as surface energies, thermal properties, and mechanical properties using the *matcalc* software. Advancements in machine learning interatomic potentials have made such simulations extremely fast and simple.


More information about the theory of machine learned potentials can be found in a previous video: https://youtu.be/tWeBrPTrSDE

In this tutorial, we'll focus on hands-on practical running of simulations. We will complete the following tasks:

1. Optimize structural parameters and calculate the total energy of GaAs
    - Repeat the calculation with a different interatomic potential / potential energy surface from matgl
    - Repeat the calculation with a different potential from the internet
2. Calculate surface energy of Si(111)
3. Calculate the phonon dispersion of Si
4. Calculate the elastic tensors of 25 different materials with parallelization

Let's get started!

# Installation of packages

The first step is to install all required packages via ``pip``. We need:

* the core ``matcalc`` library

Additionally, it is highly recommended (and necessary for this tutorial) to install:
* the ``matgl`` library, which provides some universal potential for us to use
* the ``seekpath`` library, which is needed to calculate phonon band structures

Finally, it is optional to install:
* the ``crystal_toolkit`` library, which helps with visualization in the notebook

In [ ]:
!pip install matcalc

!pip install matgl
!pip install seekpath

!pip install crystal-toolkit

# Task 1. Optimize structural parameters and calculate the total energy of GaAs

One of the most common tasks in materials modeling is optimizing the structural parameters of a crystal structure and calculating its total energy following the relaxation.

As the target structure, we will load the experimental structure of GaAs from the Crystallography Open Database, or COD, from a CIF file.

If you use structures from the COD for your own work, please make sure to cite their resource: https://wiki.crystallography.net/cod/citing/


Let's start by loading and visualizing the GaAs structure. We will use the *pymatgen* code to create a ``Structure`` object.

For more details about *pymatgen* ``Structure`` objects, as well as additional ways to load crystal structures, see Part 1 of the pymatgen tutorial: https://youtu.be/b0tieiedGdg

In [ ]:
import crystal_toolkit  # for interactive visualization only; can omit if causing problems
import os
from pymatgen.core import Structure

# Get a structure we want to optimize
gaas_expt = Structure.from_file(os.path.join("structures", "GaAs.cif"))  # expt. structure from COD
print(gaas_expt)
gaas_expt

Let's now perturb the structure from its initial position. We will do this by expanding the lattice parameters as well as adding random noise to the atom positions.

In [ ]:
gaas_perturbed = gaas_expt.copy().perturb(0.2)  # perturb atom positions
gaas_perturbed.scale_lattice(gaas_expt.volume * 1.2)  # expand the lattice

print(gaas_perturbed)
gaas_perturbed

Ok, now it's time to optimize our perturbed structure using universal potentials. The first thing we need to do is to select a potential energy surface. The *matcalc* code refers to these as "calculators".

We can print out a list of all calculators bundled with the *matgl* code that we previously installed (later we'll show how to use other potentials not bundled with *matgl*).

In [ ]:
import matcalc
from matcalc.utils import UNIVERSAL_CALCULATORS

import pprint
pprint.pprint(list(UNIVERSAL_CALCULATORS))  # calculators that come with bundled with matgl

For ease of use, we can use shortcuts to get "recommended" potentials for various levels of theory used in the training data. Note that these recommended potentials might change with time, so if you need stability you should specify the potential name explicitly.

For now, we'll load the default model that is trained on DFT-PBE level data.

In [ ]:
from matcalc.utils import MODEL_ALIASES
pprint.pprint(MODEL_ALIASES)  # list all "aliased" models

In [ ]:
calculator_pbe = matcalc.load_fp("pbe")  # recommended calculator for PBE functional - note default can change over time!
# calculator_pbe = matcalc.load_fp("TensorNet-MatPES-PBE-v2025.1-PES")  # enforce calculator explicitly

Now that we have a calculator object, we can run a simulation with *matcalc*. We need to specify the type of simulation we want to do by instantiating the appropriate Property Calclulator from *matcalc*, and then call the ``calc()`` function on our desired structure.

Here, we will try to optimize our perturbed structure to see if we can recover our original cell volume and crystallographic atom positions using the structure optimization.

In [ ]:
relax_calc = matcalc.RelaxCalc(
    calculator_pbe,
    optimizer="FIRE",
    relax_atoms=True,
    relax_cell=True,
)

data = relax_calc.calc(gaas_perturbed)

Now that we have the data from the relaxation simulation, we can examine it. It contains not only the optimized structure, but also the energy of the configuration, forces on atoms, and cell stress after optimization.

In [ ]:
pprint.pprint(data)

In [ ]:
final_structure_pbe = data["final_structure"]
print(final_structure_pbe)
final_structure_pbe

Examining the structure, we will see that we got close to recovering our perfect crystallographic atom positions. 

Our cell volume is also closer to experiment than the perturbed structure; however, because the DFT-PBE training data tends to overestimate lattice parameters, the corresponding machine-learned potential also produces cell lengths slightly larger than experiment.

## Task 1b. Repeat the calculation with a different interatomic potential / potential energy surface from matgl

Now let's repeat the same structure optimization, but use a different machine-learned interatomic potential from the *matgl* library. In particular, we will use a potential trained on meta-GGA calculations (the r2SCAN method) that is typically more accurate than PBE. However, less training data is available for r2SCAN calculations.

In [ ]:
calculator_r2scan = matcalc.load_fp("r2scan")  # recommended calculator for r2scan functional

In [ ]:
relax_calc = matcalc.RelaxCalc(
    calculator_r2scan,
    optimizer="FIRE",
    relax_atoms=True,
    relax_cell=True,
)

data = relax_calc.calc(gaas_perturbed)
final_structure_r2scan = data["final_structure"]
print(final_structure_r2scan)
final_structure_r2scan

We see that this potential has better reproduced the lattice parameters of GaAs starting from the perturbed structure.

## Task 1c. Repeat the calculation with a different potential from the internet

Now, let's use a potential that is not bundled with *matgl*. The *matcalc* software can use any potential that implements an ``ASECalculator``.

For example, the MACE library distributes foundation models that implement the ASE calculator:
https://mace-docs.readthedocs.io/en/latest/guide/foundation_models.html

We can thus install the *mace-torch* library and load the corresponding calculator for use with *matcalc*.

In [ ]:
!pip install mace-torch

In [ ]:
from mace.calculators import mace_mp

calculator_macemp = mace_mp()  #  load the MACE calculator

print("-"*46)
print("Class hierarchy for the MACE calculator object")
print("-"*46)
print(type(calculator_macemp).__mro__)  # demonstrate that this implements the ASE Calculator model

It is now straightforward to run the structure optimization with MACE.

Note that this potential is also trained on PBE data, so overestimation of lattice parameters is expected.

In [ ]:
relax_calc = matcalc.RelaxCalc(
    calculator_macemp,
    optimizer="FIRE",
    relax_atoms=True,
    relax_cell=True,
)

data = relax_calc.calc(gaas_perturbed)

final_structure_mace = data["final_structure"]
print(final_structure_mace)
final_structure_mace

# Task 2. Calculate surface energy of Si

Let's now switch to a different type of simulation. Next, we will calculate the surface energy of the (111) surface of silicon. This will involve cleaving the (111) surface and comparing the per-atom energy of the surface slab to the bulk cell.

Let's start by loading the conventional structure of Si retrieved from the Crystallography Open Database.

In [ ]:
si_expt = Structure.from_file(os.path.join("structures", "Si.cif"))  # expt structure from COD
print(si_expt)
si_expt

Next, we will create the slab cell that contains the Si(111) surface.

Note that multiple slabs can be created depending on the details of the surface cleaving, which may result in different surface energies. The ``calc_slabs()`` function has various options that can be used to control the slab properties, but for now we will just use defaults.

In [ ]:
surface_calc = matcalc.SurfaceCalc(calculator_pbe)
slab_data = surface_calc.calc_slabs(si_expt, miller_index=(1, 1, 1), inplane_supercell=(2,2))

slab_structure = slab_data[0]["slab"]
print(slab_structure)
slab_structure

With our slab structure determined, we can calculate a surface energy by comparing the slab energy to the bulk.

Surface energies are difficult to measure, but for Si(111), we are aiming to get an unreconstructed surface energy of \~0.1 eV/A^2 which is \~1.6 J/m^2.

(1) Tran, R.; Xu, Z.; Radhakrishnan, B.; Winston, D.; Sun, W.; Persson, K. A.; Ong, S. P. Surface Energies of Elemental Crystals. Sci Data 2016, 3 (1), 160080. https://doi.org/10.1038/sdata.2016.80.

See also: http://crystalium.materialsvirtuallab.org

In [ ]:
surface_data = surface_calc.calc({"bulk": si_expt, "slab": slab_structure})
print(surface_data["surface_energy"])

The basic calculation underestimates, so caution is needed when doing these kinsd of simulations quickly.

However, we will keep moving forward.

# Task 3: Calculate the phonon dispersion of Si

Next, we'll calculate the phonon dispersion of Si. The phonon dispersion provides information about crystalline vibrations which can be related to thermal conductivity, heat capacity, free energy, and other materials properties.

More information about how to interpret phonon dispersions can be found here: https://youtu.be/acT6zQbiiio

Calculating phonon properties is as easy as running the ``PhononCalc`` Property Calculator on the target structure.

We will load the primitive cell (with the fewest number of atoms in the unit cell) to make the calculations efficient as well as to have a more interpretable band structure.

In [ ]:
si_primitive = Structure.from_file(os.path.join("structures", "Si_primitive_mp-149.cif"))
si_primitive

After loading the structure, running the phonon calculation is as easy as invoking the ``PhononCalc`` property calculator using our potential energy function of choice.

In [ ]:
from matcalc import PhononCalc
phonon_calc = PhononCalc(calculator_pbe, relax_structure=True, 
                         write_band_structure=os.path.join("outputs", "si_phonon_bs.yaml"),
                         write_total_dos=os.path.join("outputs", "si_phonon_dos.dat"),
                         write_phonon=os.path.join("outputs", "si_phonon.yaml"))
data = phonon_calc.calc(si_primitive)

Let's now plot the phonon dispersion.

In [ ]:
import matplotlib.pyplot as plt

phonon_bs = data["phonon"].band_structure  # get the phonon band structure

# Plotting code below here -->
n_axes = sum(1 for c in phonon_bs.path_connections if not c)
# Create n_axes subplot(s) along 1 row
fig, axs = plt.subplots(1, n_axes, figsize=(16, 5))
# If only one axis is created, wrap it into a list
if n_axes == 1:
    axs = [axs]

    # Remove y-axis labels & ticks from all but the first subplot
for ax in axs[1:]:
    ax.set_ylabel('')
    ax.tick_params(axis='y', labelleft=False)

phonon_bs.plot(ax=axs)
fig.suptitle("Phonon Band Structure of Si", fontsize=16)
plt.subplots_adjust(wspace=0.02)  # Remove horizontal space so they almost touch
plt.show()



We can also plot the phonon density of states, or DOS.

In [ ]:
phonon_dos = data["phonon"].total_dos

fig, ax = plt.subplots(figsize=(8, 5))
phonon_dos.plot(ax=ax)
plt.title("Phonon Density of States of Si", fontsize=16)
plt.show()

Finally, let's plot the vibrational heat capacity of the material.

Note that the heat capacity is plotted in J/mol*K, where we are referring to **mols of the formula unit / molecule in the unit cell**. Since our unit cell contained 2 atoms of Si, this is the heat capacity per 2 mols of Si, not per mol of Si.

In [ ]:
# plot the heat capacity
import matplotlib.pyplot as plt
plt.plot(data["thermal_properties"]["temperatures"], data["thermal_properties"]["heat_capacity"])
plt.xlabel("Temperature (K)")
plt.ylabel("Heat Capacity (J/mol-K)")
plt.title("Heat Capacity vs Temperature")
plt.show()

All this thermal data can be compared, for example, to phonon data in the Materials Project: https://next-gen.materialsproject.org/materials/mp-149?material_ids=mp-149

# Task 4. Calculate 25 elastic tensors with parallelization

The *matcalc* software includes basic facilities to parallelize individual calculations over various processing cores. This is as simple as replacing the ``calc()`` function in a Property Calculator with the ``calc_many()`` function, and providing a list of ``Structure`` objects instead of a single object.

Let's use this functionality to calculate the elastic tensors of 25 materials in parallel.

Let's first load 25 structures.

In [ ]:
import glob

# Find all .cif files in the "cifs_mp" folder
cif_files = glob.glob("structures_batch/*.cif")  # folder contains assorted crystal structures from Materials Project
structures = []
for cif_file in cif_files:
    structures.append(Structure.from_file(cif_file))
    
print(len(structures))

We can now use the ``calc_many()`` function of ``ElasticityCalc`` to calculate elastic moduli of all these materials. Note that casting the result to a list forces the calculatons to take place and ``njobs=-1`` tries to use all cores on your processor.

We'll time how long it takes to calculate 25 elastic tensors using the ``%%time`` magic command.

In [ ]:
%%time

from matcalc import ElasticityCalc
elastic_calc = ElasticityCalc(calculator_pbe, relax_structure=True)
elastic_data = list(elastic_calc.calc_many(structures, n_jobs=-1))

Our 25 calculations are complete - let's now print out our results! Note that each item contains more data, we will just print out various elastic moduli instead of the full elastic tensor.

In [ ]:
multiplier_GPa = 160.2176621  # unit conversions for eV/A^3 to GPa

for idx, data_item in enumerate(elastic_data):
    print(cif_files[idx])
    print("--------")
    print(f"Bulk modulus: {data_item['bulk_modulus_vrh'] * multiplier_GPa} GPa")
    print(f"Shear modulus: {data_item['shear_modulus_vrh'] * multiplier_GPa} GPa")
    print("\n")

These values can be compared against calculation in the Materials Project, for example: https://next-gen.materialsproject.org/materials/mp-149

You can also get the full 3x3x3x3 (``C_ijkl``) elastic tensor (in eV/A^3) for any material.

In [ ]:
print(cif_files[0])
print(f"Elastic tensor: {elastic_data[0]['elastic_tensor']}")

# Other functionalities

The *matcalc* code contains other functionalities, including:

* additional property calculators, like nudged elastic band (NEB) activation energies, third order phonons, and molecular dynamics runs
* basic workflow capabilities, such as defining a chain of calculators that can be re-used.

To learn these functionalities, please check the official documentation at:
* https://github.com/materialsvirtuallab/matcalc
* https://matcalc.ai

![NEB calc](graphics/neb.png "NEB calculation")
